In [1]:
import sqlite3
import os


my_database = "./test5.db"
homelessusacsv = "./2007-2016-Homelessnewss-USA.csv"
popbystatecsv = "./Population-by-state.csv"

conn = sqlite3.connect(my_database)

print ("Opened database successfully");

Opened database successfully


In [2]:
import csv
import sqlite3

my_database = "./test5.db"
homelessusacsv = "./2007-2016-Homelessnewss-USA.csv"
popbystatecsv = "./Population-by-state.csv"

table = """CREATE TABLE
IF NOT EXISTS homeless_info (
 id integer PRIMARY KEY,
 year integer NOT NULL,
 state text NOT NULL,
 coc_number text NOT NULL,
 coc_name text NOT NULL,
 measures text NOT NULL,
 total integer NOT NULL 
);"""

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

def homelessness(csv_path):
    try:
        with open(csv_path, "r") as csv_file:
            db = create_connection(my_database)
            cursor = db.cursor()
            cursor.execute(table)
            cursor.close()
            cursor = db.cursor()
            reader = csv.reader(csv_file)
            rows = list(reader)[1:]
            ins = 'insert into homeless_info (year, state, coc_number, coc_name, measures, total) values(?, ?, ?, ?, ?, ?)'
            for i in rows:
                i[5] = ''.join(x for x in i[5] if x.isdigit())
                cursor.execute(ins, i)
            db.commit()    
            db.close()
    except Exception:
        db.rollback()
        db.close()
        raise       
homelessness(homelessusacsv)

table = """CREATE TABLE
IF NOT EXISTS pop_by_state (
 id text PRIMARY KEY,
 id2 integer NOT NULL,
 geo text textNULL,
 census text NOT NULL,
 est_base text NOT NULL,
 a2010 integer NOT NULL,
 a2011 integer NOT NULL,
 a2012 integer NOT NULL,
 a2013 integer NOT NULL,
 a2014 integer NOT NULL,
 a2015 integer NOT NULL,
 a2016 integer NOT NULL
);"""

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

def pop_by_state(csv_path):
    try:
        with open(csv_path, "r") as csv_file:
            db = create_connection(my_database)
            cursor = db.cursor()
            cursor.execute(table)
            cursor.close()
            cursor = db.cursor()
            reader = csv.reader(csv_file)
            rows = list(reader)[2:]
            ins = 'insert into pop_by_state (id, id2, geo, census, est_base, a2010, a2011, a2012, a2013, a2014, a2015, a2016) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
            for i in rows:
                cursor.execute(ins, i)
            db.commit()    
            db.close()
    except Exception:
        db.rollback()
        db.close()
        raise
pop_by_state(popbystatecsv)




In [4]:
import sqlite3
import bokeh
import pandas as pd

my_database = "./test5.db"
homelessusacsv = "./2007-2016-Homelessnewss-USA.csv"
popbystatecsv = "./Population-by-state.csv"

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
def graph_by_year(sql):
    db = create_connection(my_database)
    try:
        cursor = db.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
#         print(type(rows))
#         for row in rows:
#             print(row)
        cursor.close()
        db.close()
        return rows
    except Exception:
        db.rollback()
        db.close()
        raise
graph_by_year("select year, sum(total) as total from homeless_info where measures = 'Total Homeless' group by year order by year")        
#Sql Statement

#Do this for each value you're getting COC Code

def GetData( p, listOfCities):
    for coc in listOfCities:
        dataset = graph_by_year("select year, sum(total) as total from homeless_info where measures = 'Total Homeless' and coc_name = '{0}'group by year order by year".format(coc))
        timevalues = [x[0] for x in dataset]
        totalhomeless = [x[1] for x in dataset]

        result = []
        for item in timevalues:
            result.append(int(item[-4:]))
        p.line(x=result, y=totalhomeless, line_width=2)
        
listOfCities = ['Detroit CoC', 'Texas Balance of State (BoS) CoC', 'Houston, Pasadena, Conroe/Harris, Ft. Bend, Montgomery, Counties CoC', 'San Bernardino City & County CoC', 'Georgia Balance of State CoC']



    
dataset = graph_by_year("select year, sum(total) as total from homeless_info where measures = 'Total Homeless' group by year order by year")
timevalues = [x[0] for x in dataset]
totalhomeless = [x[1] for x in dataset]

result = []
for item in timevalues:
    result.append(int(item[-4:]))

from bokeh.plotting import figure, output_file, show

output_file("AllHomeless.html")

p = figure(plot_width=400, plot_height=400)

GetData(p, listOfCities)

#Add aline 
# add a line renderer
# p.line(x=result, y=totalhomeless, line_width=2)

show(p)

cocrange = pd.read_sql_query("select year, coc_name, sum(total) as total from homeless_info where measures ='Total Homeless' group by year, coc_name order by year", sqlite3.connect(my_database))

In [5]:
pivotcoc = cocrange.pivot(index="coc_name", columns="year", values="total")
pivotcoc["change"] = pivotcoc["1/1/2007"] - pivotcoc["1/1/2016"]
pivotcoc.sort_values(by = "change", ascending = False)

year,1/1/2007,1/1/2008,1/1/2009,1/1/2010,1/1/2011,1/1/2012,1/1/2013,1/1/2014,1/1/2015,1/1/2016,change
coc_name,,,,,,,,,,,
Detroit CoC,18062.0,18062.0,3694.0,2812.0,3138.0,3115.0,2789.0,2755.0,2597.0,2335.0,15727.0
Texas Balance of State (BoS) CoC,12958.0,13298.0,15327.0,14827.0,14643.0,12764.0,10128.0,9943.0,7016.0,6048.0,6910.0
"Houston, Pasadena, Conroe/Harris, Ft. Bend, Montgomery, Counties CoC",10363.0,10363.0,7576.0,6368.0,8471.0,7187.0,6359.0,5308.0,4609.0,4031.0,6332.0
San Bernardino City & County CoC,6969.0,6969.0,2026.0,2026.0,2825.0,2532.0,2321.0,2315.0,2149.0,1887.0,5082.0
Georgia Balance of State CoC,10255.0,9340.0,9941.0,9750.0,11168.0,11105.0,7651.0,7577.0,5797.0,5575.0,4680.0
Tampa/Hillsborough County CoC,6483.0,6483.0,7473.0,7473.0,7336.0,7419.0,1909.0,1944.0,1931.0,1817.0,4666.0
Los Angeles City & County CoC,47862.0,47862.0,33243.0,33243.0,34622.0,31553.0,35524.0,34393.0,41174.0,43854.0,4008.0
Austin/Travis County CoC,5281.0,3451.0,2641.0,2087.0,2362.0,2244.0,2090.0,1987.0,1832.0,2138.0,3143.0
Metropolitan Denver Homeless Initiative,8698.0,8482.0,8752.0,8752.0,4809.0,6358.0,6316.0,6621.0,5335.0,5728.0,2970.0


In [ ]:

import numpy as np

from bokeh.plotting import figure, show, output_file

x = np.linspace(0.1, 5, 100)

p = figure(title="Top 5 CoC Homeless Reducers", y_axis_type="log",
           y_range=(0.001, 10**22))

p.line(x, np.sqrt(x), legend="Detroit CoC",
       line_color="tomato", line_dash="dotdash")

p.line(x, np.sqrt(x), legend="Texas Balance of State (BoS) CoC",
       line_color="tomato", line_dash="dotdash")

p.line(x, 10**x, legend="Houston, Pasadena, Conroe/Harris, Ft. Bend, Montgomery, Counties CoC",
       line_color="gold", line_width=2)

p.line(x, x**x, legend="San Bernardino City & County CoC",
       line_dash="dotted", line_color="indigo", line_width=2)

p.line(x, 10**(x**2), legend="Georgia Balance of State CoC",
       line_color="coral", line_dash="dashed", line_width=2)

p.legend.location = "top_left"

p.xaxis.axis_label = 'Domain'
p.yaxis.axis_label = 'Values (log scale)'

output_file("topfivereducers.html", title="Top 5 CoC Homeless Reducers")

show(p)  # open a browser